## imports, constants, basic functions

In [22]:
from os import environ
import datetime, re
from functools import partial
from dataflows import (Flow, checkpoint, load, printer, 
                       update_resource, join, filter_rows, dump_to_path, set_type)
from datapackage_pipelines_knesset.common_flow import load_knesset_data as _load_knesset_data
from bill_dates import init_knessetdates, load_knessetdate_row, calc_knessetdates_range, to_date


BILL_ID_COLUMN = 'מזהה ייחודי של הצעות חוק'
USE_DATA = False


if USE_DATA:
    environ['KNESSET_PIPELINES_DATA_PATH'] = '../knesset-data-pipelines/data'
    load_knesset_data = partial(_load_knesset_data, use_data=True)
else:
    load_knesset_data = _load_knesset_data


def set_dates_any(package):
    for resource in package.pkg.descriptor['resources']:
        for field in resource['schema']['fields']:
            if field['type'] == 'datetime' or field['type'] == 'date':
                if field['format'].startswith('%'):
                    field['format'] = 'any'
    yield package.pkg
    yield from package
    

## Load source data

In [32]:
SOURCE_DATA_CHECKPOINT_NAME = 'hesderim_bill_splits_source_data'

!{'rm -rf .checkpoints/%s' % SOURCE_DATA_CHECKPOINT_NAME}

Flow(
    load('טבלת חוקי הסדרים - 10.10.18 (1).csv', name='hesderim_bill_ids'),
    load_knesset_data('knesset/kns_knessetdates/datapackage.json', resources=['kns_knessetdates']),
    load_knesset_data('bills/kns_bill/datapackage.json', resources=['kns_bill']),
    update_resource('kns_bill', name='kns_bill_0', path='kns_bill_0.csv'),
    load_knesset_data('bills/kns_billsplit/datapackage.json', resources=['kns_billsplit']),
    load_knesset_data('knesset/kns_status/datapackage.json', resources=['kns_status']),
    load_knesset_data('committees/kns_committeesession/datapackage.json', resources=['kns_committeesession']),
    load_knesset_data('committees/kns_cmtsessionitem/datapackage.json', resources=['kns_cmtsessionitem']),
    filter_rows(equals=[{'ItemTypeID': 2}], resources=['kns_cmtsessionitem']),
    load_knesset_data('plenum/kns_plenumsession/datapackage.json', resources=['kns_plenumsession']),
    load_knesset_data('plenum/kns_plmsessionitem/datapackage.json', resources=['kns_plmsessionitem']),
    filter_rows(equals=[{'ItemTypeID': 2}], resources=['kns_plmsessionitem']),
    load_knesset_data('bills/kns_bill/datapackage.json', resources=['kns_bill']),
    set_dates_any,
    checkpoint(SOURCE_DATA_CHECKPOINT_NAME),
).process()[1]

loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/knesset/kns_knessetdates/datapackage.json
loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/bills/kns_bill/datapackage.json
loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/bills/kns_billsplit/datapackage.json
loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/knesset/kns_status/datapackage.json
loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/committees/kns_committeesession/datapackage.json
loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/committees/kns_cmtsessionitem/datapackage.json
loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/plenum/kns_plenumsession/datapackage.json
loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/plenum/kns_plmsessionitem/datapackage.json
loading from url: https://storage.googleapis.com/knesset

DEBUG   :Starting new HTTPS connection (1): storage.googleapis.com:443
DEBUG   :https://storage.googleapis.com:443 "GET /knesset-data-pipelines/data/knesset/kns_knessetdates/datapackage.json HTTP/1.1" 200 2037
DEBUG   :Starting new HTTPS connection (1): storage.googleapis.com:443
DEBUG   :https://storage.googleapis.com:443 "GET /knesset-data-pipelines/data/knesset/kns_knessetdates/datapackage.json HTTP/1.1" 200 2037
DEBUG   :Starting new HTTPS connection (1): storage.googleapis.com:443
DEBUG   :https://storage.googleapis.com:443 "GET /knesset-data-pipelines/data/bills/kns_bill/datapackage.json HTTP/1.1" 200 6234
DEBUG   :Starting new HTTPS connection (1): storage.googleapis.com:443
DEBUG   :https://storage.googleapis.com:443 "GET /knesset-data-pipelines/data/bills/kns_bill/datapackage.json HTTP/1.1" 200 6234
DEBUG   :Starting new HTTPS connection (1): storage.googleapis.com:443
DEBUG   :https://storage.googleapis.com:443 "GET /knesset-data-pipelines/data/bills/kns_billsplit/datapackage

checkpoint saved: hesderim_bill_splits_source_data


{'count_of_rows': 261800,
 'bytes': 77212249,
 'hash': '91c306015f74492bf7707dc51089d970',
 'dataset_name': None}

## Join Hesderim data

In [33]:
def hesderim_bill_ids():
    
    def step(rows):
        if rows.res.name == 'hesderim_bill_ids':
            yield from ({'hesderim_bill_id': int(row[BILL_ID_COLUMN])}
                        for row in rows)
        else:
            yield from rows
    
    return Flow(
        step,
        update_resource('hesderim_bill_ids', schema={'fields': [{'name': 'hesderim_bill_id', 'type': 'integer'}]})
    )
    
    
def hesderim_bill_splits():
    hesderim_bill_ids = set()
    
    def step(rows):
        if rows.res.name == 'hesderim_bill_ids':
            for row in rows:
                hesderim_bill_ids.add(row['hesderim_bill_id'])
                yield row
        elif rows.res.name == 'kns_billsplit':
            for row in rows:
                if row['MainBillID'] in hesderim_bill_ids:
                    yield {'hesderim_bill_id': int(row['MainBillID']),
                           'bill_id': int(row['SplitBillID'])}
        else:
            yield from rows
    
    return Flow(
        step,
        update_resource('kns_billsplit', name='hesderim_bill_splits', path='hesderim_bill_splits.csv',
                        schema={'fields': [{'name': 'hesderim_bill_id', 'type': 'integer'},
                                           {'name': 'bill_id', 'type': 'integer'}]})
    )


def hesderim_bills_committee_sessions():    
    return Flow(
        join(source_name='kns_committeesession', source_key=['CommitteeSessionID'], 
             target_name='kns_cmtsessionitem', target_key=['CommitteeSessionID'], 
             fields={'KnessetNum': {}, 'StartDate': {}, 'CommitteeID': {}}),
        join(source_name='kns_status', source_key=['StatusID'], source_delete=False,
             target_name='kns_cmtsessionitem', target_key=['StatusID'],
             fields={'StatusDesc': {'name': 'Desc'}}),
        join(source_name='hesderim_bill_splits', source_key=['bill_id'], source_delete=False,
             target_name='kns_cmtsessionitem', target_key=['ItemID'],
             fields={'hesderim_bill_id': {}}),
        filter_rows(not_equals=[{'hesderim_bill_id': None}], resources=['kns_cmtsessionitem']),
    )


def hesderim_bills_plenum_sessions():
    return Flow(
        join(source_name='kns_plenumsession', source_key=['PlenumSessionID'], 
             target_name='kns_plmsessionitem', target_key=['PlenumSessionID'], 
             fields={'KnessetNum': {}, 'StartDate': {}}),
        join(source_name='kns_status', source_key=['StatusID'], source_delete=False,
             target_name='kns_plmsessionitem', target_key=['StatusID'],
             fields={'StatusDesc': {'name': 'Desc'}}),
        join(source_name='hesderim_bill_splits', source_key=['bill_id'], source_delete=False,
             target_name='kns_plmsessionitem', target_key=['ItemID'],
             fields={'hesderim_bill_id': {}}),
        filter_rows(not_equals=[{'hesderim_bill_id': None}], resources=['kns_plmsessionitem']),
    )


def join_hesderim_kns_bill():
    return Flow(
        join(source_name='hesderim_bill_splits', source_key=['bill_id'], source_delete=False,
             target_name='kns_bill', target_key=['BillID'],
             fields={'hesderim_bill_id': {}}),
        filter_rows(not_equals=[{'hesderim_bill_id': None}], resources=['kns_bill']),
    )

JOIN_HESDERIM_DATA_CHECKPOINT_NAME = 'hesderim_bill_splits_join_hesderim_data'

!{'rm -rf .checkpoints/%s' % JOIN_HESDERIM_DATA_CHECKPOINT_NAME}

Flow(
    checkpoint(SOURCE_DATA_CHECKPOINT_NAME),
    hesderim_bill_ids(),
    hesderim_bill_splits(),
    hesderim_bills_committee_sessions(),
    hesderim_bills_plenum_sessions(),
    join_hesderim_kns_bill(),
    checkpoint(JOIN_HESDERIM_DATA_CHECKPOINT_NAME)
).process()[1]


saving checkpoint to: .checkpoints/hesderim_bill_splits_join_hesderim_data
using checkpoint data from .checkpoints/hesderim_bill_splits_source_data
checkpoint saved: hesderim_bill_splits_join_hesderim_data


{'count_of_rows': 43867,
 'bytes': 9752281,
 'hash': '8932f8883c48d4379f884ca94a5380c8',
 'dataset_name': None}

In [ ]:
Flow(checkpoint(JOIN_HESDERIM_DATA_CHECKPOINT_NAME), printer(tablefmt='html', num_rows=1)).process()[1]

## Process Sessions

In [36]:
def get_plenum_session_start_date(plenum_session):
    start_date = plenum_session['StartDate'].date()
    if start_date < datetime.date(1947, 1, 1):
        m = re.findall('([0-9]+)/([0-9]+)/([0-9]+)', plenum_session['Name'])
        assert m, 'failed to find date for session {}'.format(plenum_session)
        assert len(m) == 1
        start_date = datetime.date(*map(int, reversed(m[0])))
    return start_date

def process_sessions(session_type):
    input_resource_name = {'committee': 'kns_cmtsessionitem',
                           'plenum': 'kns_plmsessionitem'}[session_type]
    output_resource_name = f'bill_{session_type}_sessions'
    first_session_field = f'{session_type}_first_session'
    last_session_field = f'{session_type}_last_session'
    num_sessions_field = f'{session_type}_num_sessions'
    bill_sessions = {}
    bill_hesderim_bill_id = {}
    
    if session_type == 'committee':
        bill_relevant_committee = {}
    elif session_type == 'plenum':
        plenum_session_dates = {}
        
    def _process_bills(rows):
        for row in rows:
            if session_type == 'committee':
                bill_relevant_committee[row['BillID']] = row['CommitteeID']
            yield row
    
    def _process_plenum_session_dates(rows):
        for row in rows:
            plenum_session_dates[row['PlenumSessionID']] = get_plenum_session_start_date(row)
            yield row
    
    def _process_sessions(rows):
        for row in rows:
            bill_sessions.setdefault(row['ItemID'], []).append(row)
            bill_hesderim_bill_id[row['ItemID']] = row['hesderim_bill_id']
            yield row
    
    def _get_bill_sessions():
        for bill_id, sessions in bill_sessions.items():
            sessions = list(sorted(sessions, key=lambda row: row['StartDate']))
            if session_type == 'plenum':
                sessions = [dict(row, StartDate=plenum_session_dates[row['PlenumSessionID']])
                            for row in sessions]
            row = {'bill_id': bill_id,
                   'hesderim_bill_id': bill_hesderim_bill_id[bill_id],
                   first_session_field: to_date(sessions[0]['StartDate']),
                   last_session_field: to_date(sessions[-1]['StartDate']),
                   num_sessions_field: len(sessions)}
            if session_type == 'committee':
                relevant_sessions = [s for s in sessions if s['CommitteeID'] == bill_relevant_committee[bill_id]]
                if len(relevant_sessions) > 0:
                    row.update(relevant_committee_first_session=to_date(relevant_sessions[0]['StartDate']),
                               relevant_committee_last_session=to_date(relevant_sessions[-1]['StartDate']),
                               relevant_committee_num_sessions=len(relevant_sessions))
                else:
                    row.update(relevant_committee_first_session=None,
                               relevant_committee_last_session=None,
                               relevant_committee_num_sessions=0)
            yield row
    
    def step(package):
        package.pkg.add_resource({'name': output_resource_name,
                                  'path': f'{output_resource_name}_sessions.csv',
                                  'schema': {'fields': [
                                      {'name': 'bill_id', 'type': 'integer'},
                                      {'name': 'hesderim_bill_id', 'type': 'integer'},
                                      {'name': first_session_field, 'type': 'date'},
                                      {'name': last_session_field, 'type': 'date'},
                                      {'name': num_sessions_field, 'type': 'integer'},                                      
                                  ] + ([{'name': 'relevant_committee_first_session', 'type': 'date'},
                                        {'name': 'relevant_committee_last_session', 'type': 'date'},
                                        {'name': 'relevant_committee_num_sessions', 'type': 'integer'},] if session_type == 'committee' else [])}})
        yield package.pkg
        for rows in package:
            if rows.res.name == 'kns_bill_0':
                yield _process_bills(rows)
            elif session_type == 'plenum' and rows.res.name == 'kns_plenumsession':
                yield _process_plenum_session_dates(rows)
            elif rows.res.name == input_resource_name:
                yield _process_sessions(rows)
            else:
                yield rows
        yield _get_bill_sessions()

    return step

PROCESS_SESSIONS_CHECKPOINT_NAME = 'hesderim_bill_splits_process_sessions_data'

!{'rm -rf .checkpoints/%s' % PROCESS_SESSIONS_CHECKPOINT_NAME}

Flow(
    checkpoint(JOIN_HESDERIM_DATA_CHECKPOINT_NAME),
    load_knesset_data('plenum/kns_plenumsession/datapackage.json', resources=['kns_plenumsession']),
    load_knesset_data('committees/kns_committee/datapackage.json'),
    set_dates_any,
    process_sessions('committee'),
    process_sessions('plenum'),
    checkpoint(PROCESS_SESSIONS_CHECKPOINT_NAME)
).process()[1]

loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/plenum/kns_plenumsession/datapackage.json
loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/committees/kns_committee/datapackage.json
saving checkpoint to: .checkpoints/hesderim_bill_splits_process_sessions_data
using checkpoint data from .checkpoints/hesderim_bill_splits_join_hesderim_data


DEBUG   :Starting new HTTPS connection (1): storage.googleapis.com:443
DEBUG   :https://storage.googleapis.com:443 "GET /knesset-data-pipelines/data/plenum/kns_plenumsession/datapackage.json HTTP/1.1" 200 1961
DEBUG   :Starting new HTTPS connection (1): storage.googleapis.com:443
DEBUG   :https://storage.googleapis.com:443 "GET /knesset-data-pipelines/data/plenum/kns_plenumsession/datapackage.json HTTP/1.1" 200 1961
DEBUG   :Starting new HTTPS connection (1): storage.googleapis.com:443
DEBUG   :https://storage.googleapis.com:443 "GET /knesset-data-pipelines/data/committees/kns_committee/datapackage.json HTTP/1.1" 200 3820
DEBUG   :Starting new HTTPS connection (1): storage.googleapis.com:443
DEBUG   :https://storage.googleapis.com:443 "GET /knesset-data-pipelines/data/committees/kns_committee/datapackage.json HTTP/1.1" 200 3820
DEBUG   :Starting new HTTPS connection (1): storage.googleapis.com:443
DEBUG   :https://storage.googleapis.com:443 "GET /knesset-data-pipelines/data/plenum/kns_

checkpoint saved: hesderim_bill_splits_process_sessions_data


{'count_of_rows': 52899,
 'bytes': 10971701,
 'hash': '4d12f5ade0b6185a43d6491f65bbe8c1',
 'dataset_name': None}

In [38]:
Flow(checkpoint(PROCESS_SESSIONS_CHECKPOINT_NAME), 
     printer(tablefmt='html', num_rows=1, resources='bill_.*_sessions')).process()[1]

using checkpoint data from .checkpoints/hesderim_bill_splits_process_sessions_data


#,bill_id (integer),hesderim_bill_id (integer),committee_first_session (date),committee_last_session (date),committee_num_sessions (integer),relevant_committee_first_session (date),relevant_committee_last_session (date),relevant_committee_num_sessions (integer)
1,88727,16633,2003-11-19,2003-12-22,2,2003-11-19,2003-12-22,2
2,88732,16633,2003-11-25,2003-12-23,2,2003-11-25,2003-12-23,2
...,,,,,,,,
220,569078,568878,2015-10-22,2015-10-22,1,2015-10-22,2015-10-22,1


#,bill_id (integer),hesderim_bill_id (integer),plenum_first_session (date),plenum_last_session (date),plenum_num_sessions (integer)
1,88726,16633,2003-11-05,2003-11-05,1
2,88727,16633,2003-11-05,2003-11-05,1
...,,,,,
52,569078,568878,2015-11-23,2015-12-14,3


{}

## main flow

In [60]:
def process_session_date_ranges():
    knessetdates = init_knessetdates()
    
    def process_kns_knessetdates(rows):
        for row in rows:
            yield row
            load_knessetdate_row(knessetdates, row)
        
    def process_bill_sessions(rows):
        for row in rows:
            row['plenum_days_with_pagra'], row['plenum_days_without_pagra'] = calc_knessetdates_range(row, 'plenum_first_session', 'plenum_last_session', knessetdates)
            row['committee_days_with_pagra'], row['committee_days_without_pagra'] = calc_knessetdates_range(row, 'committee_first_session', 'committee_last_session', knessetdates)
            row['relevant_committee_days_with_pagra'], row['relevant_committee_days_without_pagra'] = calc_knessetdates_range(row, 'relevant_committee_first_session', 'relevant_committee_last_session', knessetdates)
            all_sessions = []
            for k in ['plenum_first_session', 'committee_first_session', 'plenum_last_session', 'committee_last_session']:
                if row[k]:
                    all_sessions.append(row[k])
            all_sessions = list(sorted(all_sessions))
            if len(all_sessions) > 0:
                row['all_first_session'], row['all_last_session'] = all_sessions[0], all_sessions[-1]
                row['all_days_with_pagra'], row['all_days_without_pagra'] = calc_knessetdates_range(row, 'all_first_session', 'all_last_session', knessetdates)
            else:
                row['all_first_session'], row['all_last_session'] = None, None
                row['all_days_with_pagra'], row['all_days_without_pagra'] = 0, 0
            yield row
    
    def process_rows(rows):
        if rows.res.name == 'bill_sessions':
            yield from process_bill_sessions(rows)
        elif rows.res.name == 'kns_knessetdates':
            yield from process_kns_knessetdates(rows)
        else:
            yield from rows
            
    def add_fields(package):
        for resource in package.pkg.descriptor['resources']:
            if resource['name'] == 'bill_sessions':
                resource['schema']['fields'] += [{'name': k, 'type': v} for k, v in {
                    'plenum_days_with_pagra': 'integer',
                    'plenum_days_without_pagra': 'integer',
                    'committee_days_with_pagra': 'integer',
                    'committee_days_without_pagra': 'integer',
                    'relevant_committee_days_with_pagra': 'integer',
                    'relevant_committee_days_without_pagra': 'integer',
                    'all_first_session': 'date',
                    'all_last_session': 'date',
                    'all_days_with_pagra': 'integer',
                    'all_days_without_pagra': 'integer'
                }.items()]
        yield package.pkg
        yield from package
    
    return Flow(
        process_rows,
        add_fields
    )

FINAL_BILL_SESSIONS_CHECKPOINT_NAME = 'hesderim_bill_splits_final_bill_sessions'

!{'rm -rf .checkpoints/%s' % FINAL_BILL_SESSIONS_CHECKPOINT_NAME}

Flow(
    load(f'.checkpoints/{PROCESS_SESSIONS_CHECKPOINT_NAME}/datapackage.json', resources=['kns_committee', 'kns_knessetdates']),
    load(f'.checkpoints/{PROCESS_SESSIONS_CHECKPOINT_NAME}/datapackage.json', resources=['bill_plenum_sessions']),
    load(f'.checkpoints/{PROCESS_SESSIONS_CHECKPOINT_NAME}/datapackage.json', resources=['bill_committee_sessions']),
    load(f'.checkpoints/{JOIN_HESDERIM_DATA_CHECKPOINT_NAME}/datapackage.json', resources=['kns_bill']),
    update_resource('kns_bill', name='bill_sessions', path='bill_sessions.csv'),
    join(source_name='bill_committee_sessions', source_key=['bill_id'],
         target_name='bill_sessions', target_key=['BillID'],
         fields={'committee_first_session': {}, 
                 'committee_last_session': {},
                 'committee_num_sessions': {},
                 'relevant_committee_first_session': {}, 
                 'relevant_committee_last_session': {},
                 'relevant_committee_num_sessions': {}}),
    join(source_name='bill_plenum_sessions', source_key=['bill_id'],
         target_name='bill_sessions', target_key=['BillID'],
         fields={'plenum_first_session': {}, 
                 'plenum_last_session': {},
                 'plenum_num_sessions': {},}),
    join(source_name='kns_committee', source_key=['CommitteeID'],
         target_name='bill_sessions', target_key=['CommitteeID'],
         fields={'CommitteeName': {'name': 'Name'}}),
    process_session_date_ranges(),
    printer(num_rows=1, tablefmt='html', resources=['bill_sessions']),
    dump_to_path('data/hesderim_bill_splits')
).process()[1]

#,BillID (integer),KnessetNum (integer),Name (string),SubTypeID (integer),SubTypeDesc (string),PrivateNumber (integer),CommitteeID (integer),StatusID (integer),Number (integer),PostponementReasonID (integer),PostponementReasonDesc (string),PublicationDate (datetime),MagazineNumber (integer),PageNumber (integer),IsContinuationBill (boolean),SummaryLaw (string),PublicationSeriesID (integer),PublicationSeriesDesc (string),PublicationSeriesFirstCall (string),LastUpdatedDate (datetime),hesderim_bill_id (integer),committee_first_session (date),committee_last_session (date),committee_num_sessions (integer),relevant_committee_first_session (date),relevant_committee_last_session (date),relevant_committee_num_sessions (integer),plenum_first_session (date),plenum_last_session (date),plenum_num_sessions (integer),CommitteeName (string),plenum_days_with_pagra (integer),plenum_days_without_pagra (integer),committee_days_with_pagra (integer),committee_days_without_pagra (integer),relevant_committee_days_with_pagra (integer),relevant_committee_days_without_pagra (integer),all_first_session (date),all_last_session (date),all_days_with_pagra (integer),all_days_without_pagra (integer)
1,474507,18,"סעיף 2 (חוק מיסוי מקרקעין - הגדלת ההיצע של דירות מגורים - הוראת שעה) בהצעת חוק לצמצום הגירעון ולהתמודדות עם השפעות המשבר בכלכלה העולמית (תיקוני חקיקה), התשע""ב-2012",53,ממשלתית,,653,113,723,,,,,,,,,,,2013-05-23 18:38:01+00:00,474392,,,,,,,,,,הכספים,,,,,,,,,0,0
2,88726,16,"הצעת חוק המדיניות הכלכלית לשנת הכספים 2004 (תיקוני חקיקה), התשס""ד-2003 (מיועדת רק לצורך חלוקת הצעת החוק)",53,ממשלתית,,23,113,64,,,,,,,,,,,2013-05-23 18:38:01+00:00,16633,2003-11-19,2003-12-01,4,2003-11-19,2003-12-01,4,2003-11-05,2003-11-05,1,הכלכלה,0,0,12,12,12,12,2003-11-05,2003-12-01,26,26
...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
234,569078,20,"חוק לתיקון פקודת בריאות העם (מס' 29), התשע""ו-2015",53,ממשלתית,,928,118,951,,,2015-12-23 00:00:00+00:00,2516,303,,"שר הבריאות, בהסכמת שר האוצר ובאישור ועדת העבודה, הרווחה והבריאות של הכנסת, יקבע בתקנות כללים שיחולו על רופא, בין שכיר ובין עצמאי, שייעץ לאדם או טיפל בו במסגרת שירות מרפאה ציבורי או שירות מרפאה קהילתי, לעניין איסור המשך טיפול באותו אדם או ייעוץ לו שלא במימון ציבורי, במשך תקופה של בין ארבעה לשמונה חודשים, כפי שייקבע בתקנות. השר רשאי לקבוע בתקנות הקלות או הגבלות במקום האיסור האמור לעיל, אם מצא שניתן להסתפק בכך לשם מניעת ניגוד עניינים בין עבודתו של הרופא במסגרת שירות מרפאה ציבורי או שירות מרפאה קהילתי ובין עיסוקו כרופא שלא במימון ציבורי. שר הבריאות ידווח לוועדה פעמיים בשנה על יישומו של ההסדר המוצע ועל השלכותיו במשך שלוש שנים מיום חקיקתו. תקנות אלו יכול שיקבעו תחולה הדרגתית, לרבות בהתחשב במיקום גיאוגרפי, ובלבד שעד תום שלוש שנים מיום תחילתו של חוק זה יחולו התקנות באופן מלא על כלל הרופאים. שר הבריאות ושא האוצר רשאים להאריך, בצו, את התקופה האמורה בשנה אחת.",6071,ספר החוקים,,2018-10-07 09:19:44+00:00,568878,2015-10-22,2015-10-22,1,2015-10-22,2015-10-22,1,2015-11-23,2015-12-14,3,"ועדת העבודה, הרווחה והבריאות",21,21,0,0,0,0,2015-10-22,2015-12-14,53,53


{'count_of_rows': 384,
 'bytes': 131522,
 'hash': '40f5dfa3c2b2d77d1436a568cd271dcb',
 'dataset_name': None}

## Join to get full data including bills without any sessions